# Ladezustand und Batteriespannung
### Eine weitere Beispielanwendung von Regressionsverfahren
<img src="LadezustandBatteriespannung.png">

In [ ]:
import pandas as pd
#Daten aus: 
df = pd.read_csv("./batterieladezustand.csv")

In [ ]:
df

In [ ]:
import matplotlib.pyplot as plt
plt.ylim(-20, 100)
plt.xlim(9,13)
plt.plot(df[["Batteriespannung"]], df[["Ladezustand_Prozent"]])
plt.scatter(df[["Batteriespannung"]], df[["Ladezustand_Prozent"]])
plt.ylabel("Ladezustand der Batterie [%]")
plt.xlabel("Batteriespannung [V]")
# Es müssen noch ein paar negativ-Prozentwerte ergänzt werden, sonst funktioniert die Regression nicht so gut

### ...dieses Problem kann polynomal gelöst werden:

In [ ]:
X = df[["Batteriespannung"]].values
y = df[["Ladezustand_Prozent"]].values

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
pf = PolynomialFeatures(degree = 5, include_bias = False)
#include_bias = False  - Wenn das auf True ist, dann gibt es eine zusätzliche Spalte mit einer 1, um den Intercept zu trainieren, falls diser nicht vorhanden wäre.
pf.fit(X) #wäre hier strenggenommen nicht nötig, aber andere Prozesse verlangen das später
pf.transform(X)
X_transformed = pf.transform(X)

from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_transformed, y)

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
#sns.set_style("whitegrid", {"axes.facecolor": "#00000000", "figure.facecolor": "#00000000"})
sns.set_style("whitegrid", {})
#sns.set_context("notebook", font_scale=1)
plt.ylabel("Ladezustand der Batterie [%]")
plt.xlabel("Batteriespannung [V]")
plt.ylim(-5, 100)
plt.xlim(10.5,12.7)
plt.plot(X, y, label='Daten aus Tabelle')
plt.scatter(X, y)


values_x = [[x] for x in np.linspace(10,13, 1000)]
model_x = pf.transform(values_x)
#values_x = np.linspace(10,13, 1000) #...oder so
#model_x = [[x, x**2, x**3, x**4, x**5 ] for x in values_x] #...oder so

plt.plot(values_x, model.predict(  model_x ), label='Modell 1' )
plt.legend(loc='upper left')

...sieht aus wie die Simulation einer e-Funktion

In [ ]:
#der Bauplan für das Polynom ist:
pf.powers_
#abrufbar. Dort sind die Potenzen der 5 Spalten sichtbar. 

In [ ]:
model.coef_

In [ ]:
model.intercept_

In [ ]:
# Wie gut ist das Modell - Aber Vorsicht, das könnte Overfitting sein, da wir mit den Trainingsdaten evaluieren.
model.score(X_transformed, y)

### Formel:

$$ Ladezustand\text{[%]} = -2.62383659\mathrm{e}{+05} \cdot U \\ + 4.78502723\mathrm{e}{+04} \cdot U^{2}  \\ - 4.34515883\mathrm{e}{+03} \cdot U^{3} \\ + 1.96429579\mathrm{e}{+02} \cdot U^{4} \\- 3.53551643 \cdot U^{5} \\ + 573218.6446676$$ 

...wirkt ziemlich unnatürlich, aufwändig und der Graf sieht aus wie die Simulation einer e-Funktion

## Weitere Idee: Einfachere Formel mit einer e-Funktion

In [ ]:
X = df[["Batteriespannung"]]
y = df[["Ladezustand_Prozent"]]
y = y[0:10]
X = X[0:10]
# Das was bei Polynomen die PolynomialFeatures erzeugen, machen wir für die e-Funktion selbst:
import math
X_transformed = [[x, math.exp(x)] for x in X.values.flatten()]

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X_transformed, y)

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set_style("whitegrid", {})
plt.ylabel("Ladezustand der Batterie [%]")
plt.xlabel("Batteriespannung [V]")
plt.ylim(-5, 100)
plt.xlim(10.5,12.7)
plt.plot(X, y, label='Daten aus Tabelle')
plt.scatter(X, y)

values_x = np.linspace(10.5,12.7, 1000)
model_x = [[x, math.exp(x)] for x in values_x]
plt.plot(values_x, model.predict(  model_x ), label='Modell 2')
plt.legend(loc='upper left')


In [ ]:
# Wie gut ist das Modell??? -  Aber Vorsicht, das könnte Overfitting sein, da wir mit den Trainingsdaten evaluieren.
model.score(X_transformed, y.values)

In [ ]:
U = 12.5 #...sollte ca. 80% ergeben:
ex_U = math.e ** U
model.predict([[U, ex_U] ])

In [ ]:
print('x-Koeffizient, e hoch x - Koeffizient:', model.coef_)
print('Achsenabschnitt: ', model.intercept_)

In [ ]:
x = 12.5
ladezustand = 176.0417094 - (1.86049197e+01 * x) + (5.01145418e-04 * (math.e ** x) )
ladezustand

In [ ]:
U = 12.5
ladezustand = 176 - (18.6 * U) + (0.000501 * (math.e ** U) )
ladezustand

### Formel:

$$ Ladezustand\text{[%]} = 176 - 18.6 \cdot U + 0.000501 \cdot e^{U}$$

...diese Formel erreicht zwar nicht ganz den R-Wert des obigen Polynoms 5. Grades, aber in Anbetracht des physikalischen Hintergrunds ist diese Formel realistisch